In [75]:
import IPython.display as ipd

# Egnlish speech

In [2]:
ipd.Audio('audio/2830-3980-0043.wav')

In [3]:
!deepspeech --model deepspeech-0.9.3-models.pbmm --scorer deepspeech-0.9.3-models.scorer --audio audio/2830-3980-0043.wav

Loading model from file deepspeech-0.9.3-models.pbmm
TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85
2021-05-13 05:49:47.208385: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loaded model in 0.0158s.
Loading scorer from files deepspeech-0.9.3-models.scorer
Loaded scorer in 0.000202s.
Running inference.
experience proves this
Inference took 1.285s for 1.975s audio file.


# Chinese Language

In [7]:
ipd.Audio('common_voice_zh-TW_17367893.wav')

In [9]:
!deepspeech --model deepspeech-0.9.3-models-zh-CN.pbmm --scorer deepspeech-0.9.3-models-zh-CN.scorer --audio common_voice_zh-TW_17367893.wav

Loading model from file deepspeech-0.9.3-models-zh-CN.pbmm
TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85
2021-05-13 05:57:47.562850: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loaded model in 0.0197s.
Loading scorer from files deepspeech-0.9.3-models-zh-CN.scorer
Loaded scorer in 0.000378s.
Running inference.
Traceback (most recent call last):
  File "/usr/local/bin/deepspeech", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/deepspeech/client.py", line 164, in main
    print(ds.stt(audio))
UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-9: ordinal not in range(128)


In [120]:
!python client.py --model deepspeech-0.9.3-models-zh-CN.pbmm --scorer deepspeech-0.9.3-models-zh-CN.scorer --audio common_voice_zh-TW_17367893.wav

Loading model from file deepspeech-0.9.3-models-zh-CN.pbmm
TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85
2021-05-14 06:51:40.340687: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loaded model in 0.0127s.
Loading scorer from files deepspeech-0.9.3-models-zh-CN.scorer
Loaded scorer in 0.000276s.
Running inference.
Traceback (most recent call last):
  File "client.py", line 170, in <module>
    main()
  File "client.py", line 164, in main
    print(ds.stt(audio).encode("utf-8").decode("utf-8", "ignore"))
UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-9: ordinal not in range(128)


In [106]:
y = b'\xe4\xb8\x8a\xe5\x89\x8d\xe6\x88\x91\xe7\x9a\x84\xe4\xba\x8b\xe4\xb8\x9a\xe4\xba\xba\xe5\x91\x98\xe8\xa7\x84\xe5\x88\x92'

In [107]:
print(type(y))

<class 'bytes'>


In [108]:
print(y.decode('utf-8'))

上前我的事业人员规划


In [109]:
type(y.decode('utf-8'))

str

In [112]:
print("中文")

中文


In [114]:
print("中文".encode(encoding="utf-8").decode("utf-8"))

中文


In [132]:
import sys, wave, subprocess, shlex
import numpy as np
from timeit import default_timer as timer
from deepspeech import Model, version
try:
    from shhlex import quote
except ImportError:
    from pipes import quote
    
def convert_samplerate(audio_path, desired_sample_rate):
    sox_cmd = 'sox {} --type raw --bits 16 --channels 1 --rate {} --encoding signed-integer --endian little --compression 0.0 --no-dither - '.format(quote(audio_path), desired_sample_rate)
    try:
        output = subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
    except OSError as e:
        raise OSError(e.errno, 'SoX not found, use {}hz files or install it: {}'.format(desired_sample_rate, e.strerror))

    return desired_sample_rate, np.frombuffer(output, np.int16)

model = "deepspeech-0.9.3-models-zh-CN.pbmm"
scorer = "deepspeech-0.9.3-models-zh-CN.scorer"
audio = "common_voice_zh-TW_17367893.wav"

print('Loading model from file {}'.format(model), file=sys.stderr)
model_load_start = timer()
# sphinx-doc: python_ref_model_start
ds = Model(model)
# sphinx-doc: python_ref_model_stop
model_load_end = timer() - model_load_start
print('Loaded model in {:.3}s.'.format(model_load_end), file=sys.stderr)

desired_sample_rate = ds.sampleRate()

if scorer:
    print('Loading scorer from files {}'.format(scorer), file=sys.stderr)
    scorer_load_start = timer()
    ds.enableExternalScorer(scorer)
    scorer_load_end = timer() - scorer_load_start
    print('Loaded scorer in {:.3}s.'.format(scorer_load_end), file=sys.stderr)

fin = wave.open(audio, 'rb')
fs_orig = fin.getframerate()
if fs_orig != desired_sample_rate:
    print('Warning: original sample rate ({}) is different than {}hz. Resampling might produce erratic speech recognition.'.format(fs_orig, desired_sample_rate), file=sys.stderr)
    fs_new, audio = convert_samplerate(audio, desired_sample_rate)
else:
    audio = np.frombuffer(fin.readframes(fin.getnframes()), np.int16)

audio_length = fin.getnframes() * (1/fs_orig)
fin.close()

print('Running inference.', file=sys.stderr)
inference_start = timer()
# sphinx-doc: python_ref_inference_start
print(ds.stt(audio))
# sphinx-doc: python_ref_inference_stop
inference_end = timer() - inference_start
print('Inference took %0.3fs for %0.3fs audio file.' % (inference_end, audio_length), file=sys.stderr)

Loading model from file deepspeech-0.9.3-models-zh-CN.pbmm
Loaded model in 0.0109s.
Loading scorer from files deepspeech-0.9.3-models-zh-CN.scorer
Loaded scorer in 0.000524s.
Running inference.


上前我的事业人员规划


Inference took 4.664s for 4.488s audio file.
